In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
train_data = pd.read_csv("/content/train_int.csv")
traindatat.drop(columns=['id'], inplace=True)
tesdatant = pd.read_csv("/content/test_int.csv")

In [ ]:
train_data['review_lenght'] = np.array(list(map(len, X_train_data)))
median = train_data['review_lenght'].median()
mean = train_data['review_lenght'].mean()
mode = train_data['review_lenght'].mode()[0]

In [10]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;#]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z +_]')
STOPWORDS = set(stopwords.words('english'))

def text_prepare(text):
    """
        text: a string
       s return: modified initial string
    """
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(" ", text)
    text = BAD_SYMBOLS_RE.sub("", text)
    text = text.split()
    text = " ".join([word for word in text if not word in STOPWORDS])
    return text

In [11]:
train_int['review'] = train_int['review'].apply(text_prepare)
test_int['review'] = test_int['review'].apply(text_prepare)

In [26]:
def tfidf_features(X_train, X_val, X_test):
    """
        X_train, X_val, X_test — samples        
        return TF-IDF vectorized representation of each sample and vocabulary
    """
    # Create TF-IDF vectorizer with a proper parameters choice
    # Fit the vectorizer on the train set
    # Transform the train, test, and val sets and return the result
    tfidf_vectorizer = TfidfVectorizer(token_pattern='(\S+)', min_df=5, max_df=0.9, ngram_range=(1,2))
    tfidf_vectorizer.fit(X_train)
    X_train = tfidf_vectorizer.transform(X_train)
    X_val = tfidf_vectorizer.transform(X_val)
    X_test = tfidf_vectorizer.transform(X_test)
    return X_train, X_val, X_test, tfidf_vectorizer.vocabulary_

In [27]:
train, test = train_test_split(train_int, test_size=0.15)
X_train = train.review
X_val = test.review
X_test = test_int['review']
y_train = train.sentiment
y_val = test.sentiment

In [28]:
X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train, X_val, X_test)
tfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}

In [16]:
from sklearn.linear_model import LogisticRegression

In [29]:
X_train_tfidf

<33644x123875 sparse matrix of type '<class 'numpy.float64'>'
	with 4582858 stored elements in Compressed Sparse Row format>

In [30]:
classifier = LogisticRegression()
classifier.fit(X_train_tfidf,y_train.to_numpy())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [32]:
y_pred = classifier.predict(X_val_tfidf)

In [34]:
from sklearn.metrics import f1_score, confusion_matrix
print('F1-score: {0}'.format(f1_score(y_pred, y_val.to_numpy())))
print('Confusion matrix:')
confusion_matrix(y_pred, y_val.to_numpy())

F1-score: 0.9013289036544849
Confusion matrix:


array([[2631,  259],
       [ 335, 2713]])

In [35]:
y_sub = classifier.predict(X_test_tfidf)

In [38]:
sub = pd.read_csv("/content/submission_int.csv")

In [39]:
sub["sentiment"] = y_sub

In [40]:
sub.to_csv('/content/sub4_int.csv', index=False, columns=['id','sentiment'])